In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

!git clone https://github.com/RosettaCommons/protein_generator

In [ ]:
%cd ./protein_generator
!conda env create -f environment.yml

In [ ]:
%%bash
source activate proteingenerator
pip install -e ./

In [ ]:
!wget http://files.ipd.uw.edu/pub/sequence_diffusion/checkpoints/SEQDIFF_230205_dssp_hotspots_25mask_EQtasks_mod30.pt
!wget http://files.ipd.uw.edu/pub/sequence_diffusion/checkpoints/SEQDIFF_221219_equalTASKS_nostrSELFCOND_mod30.pt

In [ ]:
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/conditional_generation/protein_generator_tev/generation_metadata_proteingenerator_tev.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")


Existing generation metadata read in.


In [ ]:
import time
!mkdir samples


generation_command = f"""
source activate proteingenerator
python ./inference.py --num_designs 50 --out samples/scaffolding --pdb /content/drive/MyDrive/Generative_Models/conditional_generation/tev_monomer.pdb  --contigs 27,A28-33,13,A47-51,88,A140-152,15,A168-179,32,A212-221,16 --T 25 --save_best_plddt
"""
#generation_command = f"""
#source activate proteingenerator
#python ./inference.py --num_designs 50 --out samples/scaffolding --sequence XXXXXXXXXXXXXXXXXXXXXXXXXXXGHTTSLXXXXXXXXXXXXXHLFRRXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXFWKHWIQTKDGQCXXXXXXXXXXXXXXXHSASNFTNTNNYXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXWGGHKVFMVKXXXXXXXXXXXXXXXX --T 25 --save_best_plddt
#"""


batch_size = 50
meta_data = {}
meta_data['batch_id'] = str(uuid.uuid4())
meta_data['batch_size'] = str(batch_size)
meta_data['Timestamp'] = str(datetime.now())
meta_data['model'] = 'protein_generator'
meta_data['task'] = 'all_atom_pdb_generation'
meta_data['conditions'] = 'tev (monomer) scaffolding (structure) [--contigs 27,A28-33,13,A47-51,88,A140-152,15,A168-179,32,A212-221,16]'
#meta_data['conditions'] = 'tev (monomer) scaffolding (sequence) [--sequence XXXXXXXXXXXXXXXXXXXXXXXXXXXGHTTSLXXXXXXXXXXXXXHLFRRXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXFWKHWIQTKDGQCXXXXXXXXXXXXXXXHSASNFTNTNNYXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXWGGHKVFMVKXXXXXXXXXXXXXXXX]'
meta_data['gpu'] = 'T4 GPU'
start_time = time.time()
!{generation_command}
end_time = time.time()
total_job_time = end_time - start_time
meta_data['wall_time_batch'] = str(total_job_time) + " Seconds"
meta_data['wall_time_task'] = str(total_job_time/batch_size) + " Seconds (inferred)"

for filename in os.listdir("/content/protein_generator/samples"):
  if filename.endswith(".pdb"):
    meta_data['entity_id'] = str(uuid.uuid4())
    #new_name = "proteingenerator_tev_seq_" + meta_data['entity_id'] + ".pdb"
    new_name = "proteingenerator_tev_struct_" + meta_data['entity_id'] + ".pdb"
    meta_data['output_file_name'] = new_name
    metadata_entry = pd.Series(meta_data)
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(metadata_entry).T], ignore_index=True)
    cleanup_command = f"""mv /content/protein_generator/samples/{filename} /content/drive/MyDrive/Generative_Models/IL10_scaffolding/protein_generator_il10/{new_name}"""
    !{cleanup_command}

all_metadata_df.to_csv(meta_data_filepath, index=False)
print("Metadata saved. Cleaning up....")
!rm -r /content/protein_generator/samples

torch.cuda.empty_cache()